<a href="https://colab.research.google.com/github/AndreaBertoglio/MLDM/blob/master/Cartella%20Prova/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pip
import sys
#if not 'sklearn' in sys.modules.keys():
#    pip.main(['install', 'sklearn'])
#if not 'kaggle' in sys.modules.keys():
#    pip.main(['install', 'kaggle'])
import random

print("Random number with seed 2020")
# first call
random.seed(2020)

Random number with seed 2020


In [ ]:
import numpy as np
import pandas as pd
import graphviz

try:
    import xgboost
except ImportError as ex:
    print("Error: the xgboost library is not installed.")
    xgboost = None
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

# Read the data from GitHub
train = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv')


In [ ]:
#Le classi sono 1=Good 0=Disappointing
train["Quality"] = np.where(train["Quality"].str.contains("Good"), 1, 0)

In [ ]:
# pull data into target (y) and predictors (X)
#La y è la classe, cioè la Quality
train_y = train.Quality
#seleziono colonne di interesse, non metto l'ID perchè non mi interessa
predictor_cols = ['fixed.acidity','volatile.acidity','citric.acid','residual.sugar','chlorides','free.sulfur.dioxide','total.sulfur.dioxide','density','pH','sulphates','alcohol']

# La x sono gli attributi
train_X = train[predictor_cols]

# Sostituisce i missing values con la media e lo applica alle x
#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = IterativeImputer(missing_values=np.nan, max_iter=100, initial_strategy='mean')
imp = imp.fit(train_X)


# Impute our data, then train
train_X_imp = imp.transform(train_X)


In [ ]:
#Uso l'80% dei dati per train e il restante 20% per test
xTrain, xTest, yTrain, yTest = train_test_split(train_X_imp, train_y, train_size = 0.8, random_state = 0)

In [ ]:
def generateSubmission(myModel, submissionFile, description):
    # Read the test data
    test = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/test.csv')


    # Treat the test data in the same way as training data. In this case, pull same columns.
    test_X = test[predictor_cols]

    # Impute each test item, then predict
    test_X_imp = imp.transform(test_X)
    
    # Use the model to make predictions
    predicted_q = myModel.predict(test_X_imp)
    # We will look at the predicted Qualities to ensure we have something sensible.
    print(predicted_q)
    
    #submission file
    my_submission = pd.DataFrame({'Id': test.Id, 'Quality': predicted_q})
    # you could use any filename. We choose submission here
    my_submission.to_csv(submissionFile, index=False)
    
    #Submit authomatically; kaggle API authentication needed
    #!kaggle competitions submit -c mldm-classification-competition-2020 -f {submissionFile} -m '{description}'

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
xgb = xgboost.XGBClassifier()
#Range di parametri che vengono esplorati per trovare la combinazione migliore
parameter_grid = {'n_estimators': [150,200,250],
          'max_depth':[7,8,9],
          'min_samples_split': [5],
          'learning_rate': [0.1,0.2,0.5],
          'loss': ['ls']}


cross_validation = StratifiedKFold(n_splits=10)
cross_validation.get_n_splits(train_X_imp, train_y)
#Create the scoring dictionary
SCORING = {'accuracy': 'accuracy',
'balanced_accuracy': 'balanced_accuracy',
'precision': 'precision_macro',
'recall': 'recall_macro',
'f1': 'f1_macro'}

grid_search = GridSearchCV(xgb, param_grid=parameter_grid, cv=cross_validation, scoring=SCORING,return_train_score=True, refit='f1')

grid_search.fit(xTrain, yTrain)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary...
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.2, 0.5], 'loss': ['ls'],
                         'max_depth': [7, 8, 9], 'min_samples_split': [5],
                         'n_estimators': [150, 200, 250]},
             pre_dispatch='2*n_jobs', refit='f1', return_train_score=True,
             scoring={'accuracy': 'accuracy',
      

In [ ]:
grid_search.cv_results_

In [ ]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_xgb = grid_search.best_estimator_
best_xgb

Best score: 0.7815948414961221
Best parameters: {'learning_rate': 0.2, 'loss': 'ls', 'max_depth': 8, 'min_samples_split': 5, 'n_estimators': 200}


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, loss='ls', max_delta_step=0, max_depth=8,
              min_child_weight=1, min_samples_split=5, missing=None,
              n_estimators=200, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [ ]:
my_model=best_xgb
my_model.fit(train_X_imp, train_y)
my_model.score(xTest, yTest)

1.0

In [ ]:
y_pred = my_model.predict(train_X_imp)
print("Confusion Matrix:")
print(confusion_matrix(train_y, y_pred))

print("Classification Report:")
print(classification_report(train_y, y_pred))

Confusion Matrix:
[[1205    0]
 [   0 2284]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1205
           1       1.00      1.00      1.00      2284

    accuracy                           1.00      3489
   macro avg       1.00      1.00      1.00      3489
weighted avg       1.00      1.00      1.00      3489



In [ ]:
# The snippet below will retrieve the feature importances from the model and make them into a DataFrame.
feature_importances = pd.DataFrame(my_model.feature_importances_,
                                   index = train_X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
alcohol,0.239804
volatile.acidity,0.126390
free.sulfur.dioxide,0.090208
residual.sugar,0.077225
citric.acid,0.075743
pH,0.073681
fixed.acidity,0.069710
sulphates,0.066629
total.sulfur.dioxide,0.064764
chlorides,0.058642


In [ ]:
# generate a submission file
generateSubmission(my_model,'XGBoost_4.csv', "User defined decision tree evaluated with Grid Search and SCORE")

[1 1 1 ... 1 1 1]
